In [384]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
import statsmodels.api as sm
import statsmodels.formula.api as smf

import sys
sys.path.append('c:/Users/tomas/Documents/Programación/Github/Patricionog/Modulio/Readmes')
import Frampy as fr
import Graphpy as gr
import Stringpy as st

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.miscmodels.ordinal_model import OrderedModel
from statsmodels.stats.proportion import proportions_chisquare
from statsmodels.stats.diagnostic import linear_harvey_collier
from scipy import stats

Los métodos robustos disminuyen el grado en que los resultados pueden verse influenciados por violaciones de los supuestos estadísticos clásicos.

In [385]:
# Cargar df.
df = pd.read_excel('J:/My Drive/Experimento/Plantillas finales/0. Generales - Base final.xlsx')  

In [386]:
# Definir variables.
Variables_Independientes = ['genero',
                            'edad',
                            #'nacionalidad',
                            #'provincia',
                            #'e_social',
                            #'niv_educativo',
                            'voto_2019',
                            #'voto_PASO_2023',
                            #'candidato_PASO_2023',
                            'autopercep_izqder',
                            'autopercep_conpro',
                            'autopercep_perantiper',
                            'indice_positividad',
                            #'massa_ip_izqder',
                            #'massa_ip_conpro',
                            #'bullrich_ip_izqder',
                            #'bullrich_ip_conpro',
                            #'schiaretti_ip_izqder',
                            #'schiaretti_ip_conpro',
                            #'milei_ip_izqder',
                            #'milei_ip_conpro',
                            #'bregman_ip_izqder',
                            #'bregman_ip_conpro',
                            'indice_progresismo',
                            'indice_conservadurismo'
                            ]

Variables_Dependientes = ['cercania_Massa', 'cercania_Milei', 
                          'cercania_Bregman', 'cercania_Bullrich', 'cercania_Schiaretti']

In [387]:
# Rellenar valores NaN con 0 para todo el DataFrame.
df.fillna(0, inplace=True)

In [388]:
# Cortar df.
df = df[Variables_Dependientes + Variables_Independientes]

In [389]:
# Convertir variables categóricas a dummies.
for Column in Variables_Independientes:
    if df[Column].dtype == 'object':
        df = fr.Create_Dummy_Variables(df, Column, Drop_First = False, Group_Others = True, Remove_Others = False, 
                                    Threshold = 0.05, Name_Other_Column = f"{Column}_Others", Name_Columns_Style = None, Separator=None)

In [390]:
# Reemplazar espacios en los nombres de las columnas con guiones bajos.
df.columns = df.columns.str.replace(' ', '_')

df.head(5)

,cercania_Massa,cercania_Milei,cercania_Bregman,cercania_Bullrich,cercania_Schiaretti,edad,autopercep_izqder,autopercep_conpro,autopercep_perantiper,indice_positividad,indice_progresismo,indice_conservadurismo,genero_femenino,genero_masculino,genero_Others,voto_2019_Alberto_Fernandez,voto_2019_Mauricio_Macri,voto_2019_Nicolás_Del_Caño,voto_2019_no_vote,voto_2019_Others
0,1,2,1,4,5,67,6,8,10,15,2.000000,4.111111,True,False,0,False,True,False,False,0
1,1,3,1,1,4,54,8,2,3,15,3.555556,2.700000,False,True,0,False,False,False,False,1
2,1,1,1,5,1,68,10,10,10,15,3.000000,2.600000,False,True,0,False,True,False,False,0
3,1,1,1,3,1,50,10,2,10,13,2.800000,3.444444,True,False,0,False,True,False,False,0
4,1,3,1,5,3,37,6,5,6,13,2.400000,4.000000,True,False,0,False,True,False,False,0


In [391]:
# Convertir columnas de objeto a numéricas.
def Convert_Object_Columns_In_Numerics(df):

    """
    Convert boolean columns to numeric (0 and 1).
    
    Args:
        df (pd.DataFrame): The dataframe containing the data.
    
    Returns:
        pd.DataFrame: The dataframe with boolean columns converted to integers.
        
    """
        
    # Convertir columnas booleanas a numéricas (0 y 1).
    for Column in df.columns:
        if df[Column].dtype == 'bool':
            df[Column] = df[Column].astype(int)
    
    return df

df = Convert_Object_Columns_In_Numerics(df)

In [392]:
# Redefinir las variables independientes.
Variables_Independientes = [Column for Column in df.columns if Column not in Variables_Dependientes]
Variables_Independientes.remove('genero_Others')
Variables_Independientes.remove('genero_masculino')
Variables_Independientes.remove('voto_2019_Others')
Variables_Independientes.remove('voto_2019_no_vote')
Variables_Independientes.remove('voto_2019_Nicolás_Del_Caño')

In [393]:
# Escalar variables independientes.
from sklearn.preprocessing import StandardScaler

Scaler = StandardScaler()
df[Variables_Independientes] = Scaler.fit_transform(df[Variables_Independientes])

In [394]:
def Generalized_Ordinal_Regression(df: pd.DataFrame, V_Ind: list, V_Dep: str, Summary: bool = False) -> tuple:
    """
    Perform an ordinal regression using OrderedModel from statsmodels.
    
    Args:
        df (pd.DataFrame): The dataframe containing the data.
        V_Ind (list): List of independent variable names.
        V_Dep (str): Name of the dependent variable.
        Summary (bool): Whether to print the summary of the regression model.
        
    Returns:
        tuple: A tuple containing:
            - A list of significant variables based on p-values.
            - A DataFrame with p-values and odds ratios for each variable.
    """
    
    # Definir las variables dependientes e independientes.
    Y = df[V_Dep]
    X = df[V_Ind]

    # Ajustar el modelo de regresión ordinal.
    Model = OrderedModel(Y, X, distr='logit')  # Se puede usar 'probit' en lugar de 'logit' si se prefiere.
    Results = Model.fit(method='bfgs', maxiter=1000)

    # Imprimir el resumen si se solicita.
    if Summary:
        print(Results.summary())

    # Generar una lista de variables con p-valor menor que 0.05.
    Significant_Variables = Results.pvalues[Results.pvalues < 0.05].index.tolist()

    # Calcular las razones de probabilidades.
    Odds_Ratios = np.exp(Results.params)  # Calcular los coeficientes exponenciados.

    # Crear un DataFrame para contener p-valores y razones de probabilidades.
    Summary_Statistics = pd.DataFrame({
        'Variable': Results.params.index,
        'P_Value': Results.pvalues,
        'Odds_Ratio': Odds_Ratios
    })

    return Significant_Variables, Summary_Statistics

In [395]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

def Check_Multicollinearity(X: pd.DataFrame):

    """
    Calculate VIF for each feature in the DataFrame and return variables with high multicollinearity.
    
    Args:
        X (pd.DataFrame): The dataframe containing the independent variables.
        
    Returns:
        None

    """
    
    # Calcular el VIF para cada característica en el DataFrame y devolver variables con alta multicolinealidad.
    VIF_Data = pd.DataFrame()
    VIF_Data["Feature"] = X.columns
    VIF_Data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    # Mostrar características con VIF > 10 (consideradas problemáticas).
    print(VIF_Data[VIF_Data["VIF"] > 10])
    
Check_Multicollinearity(df[Variables_Independientes])

Empty DataFrame
Columns: [Feature, VIF]
Index: []


In [396]:
for Variable_Dependiente in Variables_Dependientes: 
    V_Dependientes_Para_Comparar = Variables_Independientes

    print('----------------------------------------------------------------')
    print(f'Variable dependiente: {Variable_Dependiente}.')
    print('----------------------------------------------------------------')

    Significativas, Summary_Statistics = Generalized_Ordinal_Regression(df, Variables_Independientes, Variable_Dependiente)

    Variables_To_Remove = ['const', '1/2', '2/3', '3/4', '4/5']
    for Variable in Variables_To_Remove:
        if Variable in Significativas:
            Significativas.remove(Variable)

    # Verificar supuestos y eliminar variables insignificantes iterativamente.
    while sorted(Significativas) != sorted(V_Dependientes_Para_Comparar):
        V_Dependientes_Para_Comparar = Significativas
        Significativas, Summary_Statistics = Generalized_Ordinal_Regression(df, V_Dependientes_Para_Comparar, Variable_Dependiente)

        for Variable in Variables_To_Remove:
            if Variable in Significativas:
                Significativas.remove(Variable)
    
    print(f'Variables significativas:')
    for Variable in Significativas:
        Odds_Ratio_Value = Summary_Statistics.loc[Summary_Statistics['Variable'] == Variable, 'Odds_Ratio'].values[0]
        P_Value = Summary_Statistics.loc[Summary_Statistics['Variable'] == Variable, 'P_Value'].values[0]
        print(f'Variable: {Variable}, Odds Ratio: {Odds_Ratio_Value:.2f}, P-Value: {P_Value:.2e}')

----------------------------------------------------------------
Variable dependiente: cercania_Massa.
----------------------------------------------------------------
Optimization terminated successfully.
         Current function value: 1.162419
         Iterations: 26
         Function evaluations: 27
         Gradient evaluations: 27
Optimization terminated successfully.
         Current function value: 1.162779
         Iterations: 26
         Function evaluations: 27
         Gradient evaluations: 27
Optimization terminated successfully.
         Current function value: 1.163420
         Iterations: 24
         Function evaluations: 25
         Gradient evaluations: 25
Variables significativas:
Variable: edad, Odds Ratio: 0.87, P-Value: 5.30e-04
Variable: autopercep_perantiper, Odds Ratio: 0.41, P-Value: 2.13e-54
Variable: indice_positividad, Odds Ratio: 1.70, P-Value: 5.34e-37
Variable: indice_progresismo, Odds Ratio: 0.83, P-Value: 8.27e-04
Variable: indice_conservadurismo, Odd

In [398]:
def Chequeo_Proporcionalidad_Logitos(DataFrame: pd.DataFrame, Variable_Ordinal: str, Variables_Independientes: list) -> None:
    
    """
    Check the proportional odds assumption for an ordinal regression model.
    
    Args:
        DataFrame (pd.DataFrame): The DataFrame containing the data.
        Variable_Ordinal (str): The ordinal dependent variable name.
        Variables_Independientes (list): List of independent variable names.
    
    Returns:
        None

    """
    
    # Verificar que la variable dependiente y las variables independientes estén en el DataFrame.
    if Variable_Ordinal not in DataFrame.columns:
        print(f"Error: La variable ordinal '{Variable_Ordinal}' no está en el DataFrame.")
        return
    
    for Variable in Variables_Independientes:
        if Variable not in DataFrame.columns:
            print(f"Error: La variable independiente '{Variable}' no está en el DataFrame.")
            return
    
    # Ajustar el modelo ordinal.
    Formula = f"{Variable_Ordinal} ~ " + " + ".join(Variables_Independientes)
    Model = smf.mnlogit(formula=Formula, data=DataFrame).fit()
    
    # Obtener los coeficientes.
    Coeficientes = Model.params
    
    # Mostrar los coeficientes.
    print("Coeficientes del modelo por categoría:")
    print(Coeficientes)

    # Chequear la proporcionalidad.
    # En este caso, solo mostramos un ejemplo simple de cómo se podría verificar la proporcionalidad.
    # Es recomendable utilizar pruebas estadísticas más complejas como la prueba de Brant para un análisis más riguroso.
    Categorias = Coeficientes.index.unique(level=0).tolist()
    
    # Comparar coeficientes entre categorías.
    for i in range(1, len(Categorias)):
        Coef1 = Coeficientes.loc[Categorias[i]]
        Coef0 = Coeficientes.loc[Categorias[0]]
        
        Igualdad = np.all(np.isclose(Coef1, Coef0))
        
        if Igualdad:
            print(f"Se cumple el supuesto de proporcionalidad de los logitos para la categoría '{Categorias[i]}'.")
        else:
            print(f"No se cumple el supuesto de proporcionalidad de los logitos para la categoría '{Categorias[i]}'.")
            
            
# Uso de la función.
Chequeo_Proporcionalidad_Logitos(df, 'cercania_Massa', Variables_Independientes)


Optimization terminated successfully.
         Current function value: 1.147268
         Iterations 7
Coeficientes del modelo por categoría:
                                    0         1         2         3
Intercept                   -0.300459  0.457798 -0.453242 -1.605907
edad                        -0.054141 -0.035572 -0.258160 -0.198441
autopercep_izqder           -0.054034  0.047870  0.325689  0.257631
autopercep_conpro            0.168186  0.144181  0.219282  0.030991
autopercep_perantiper       -0.331394 -0.756473 -1.360467 -1.915812
indice_positividad           0.201402  0.597940  0.795561  1.201385
indice_progresismo          -0.106516 -0.321700 -0.368163 -0.578795
indice_conservadurismo       0.389348  0.543157  0.708522  0.479186
genero_femenino              0.015056  0.054551  0.105331  0.127044
voto_2019_Alberto_Fernandez  0.442888  0.755972  1.032069  1.035693
voto_2019_Mauricio_Macri    -0.162995 -0.117590 -0.303121 -0.264410
No se cumple el supuesto de proporcionalida

In [405]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

def Normalize_Variables(DataFrame: pd.DataFrame, Variables: list) -> pd.DataFrame:
    """
    Normalize specified variables in the DataFrame to the range [0, 1].
    
    Args:
        DataFrame (pd.DataFrame): The DataFrame containing the data.
        Variables (list): List of variable names to normalize.
        
    Returns:
        pd.DataFrame: The DataFrame with normalized variables.
    """
    # Create a copy to avoid modifying the original DataFrame
    Normalized_DataFrame = DataFrame.copy()
    
    # Initialize the scaler
    Scaler = MinMaxScaler()
    
    # Normalize the specified variables
    Normalized_DataFrame[Variables] = Scaler.fit_transform(Normalized_DataFrame[Variables])
    
    return Normalized_DataFrame

# Suponiendo que df es tu DataFrame original
Variables_To_Normalize = [Variable for Variable in Variables_Independientes if Variable in df.columns]
Normalized_DataFrame = Normalize_Variables(df, Variables_To_Normalize)


In [406]:
import numpy as np
import pandas as pd
import statsmodels.api as Sm
import statsmodels.formula.api as Smf
from scipy import stats

def Brant_Test(DataFrame: pd.DataFrame, Dependent_Variable: str, Independent_Variables: list) -> pd.DataFrame:
    """
    Perform the Brant test for proportional odds assumption.
    
    Args:
        DataFrame (pd.DataFrame): The DataFrame containing the data.
        Dependent_Variable (str): The ordinal dependent variable name.
        Independent_Variables (list): List of independent variable names.
        
    Returns:
        pd.DataFrame: A DataFrame containing the Brant test results.
    """
    
    # Fit the ordinal regression model
    Formula = f"{Dependent_Variable} ~ " + " + ".join(Independent_Variables)
    Model = Smf.mnlogit(formula=Formula, data=DataFrame).fit()
    
    # Get the predicted probabilities
    Predicted_Probabilities = Model.predict(DataFrame[Independent_Variables])
    
    # Create a list to hold results
    Results_List = []
    
    # Iterate over each independent variable
    for Variable in Independent_Variables:
        # Calculate the score test statistic
        Score = np.sum(Predicted_Probabilities * (DataFrame[Dependent_Variable] - Predicted_Probabilities), axis=0)
        
        # Compute the score for the current variable
        Variable_Index = Independent_Variables.index(Variable)
        Variable_Score = Score[Variable_Index]
        
        # Compute the variance for the predicted probabilities of the specific variable
        Variance = np.var(Predicted_Probabilities, axis=0)
        
        # Check if the Variable_Index is within the bounds of Variance
        if Variable_Index < len(Variance):
            # Calculate the Z-Statistic
            Z_Statistic = Variable_Score / np.sqrt(Variance[Variable_Index])
        else:
            Z_Statistic = np.nan  # Assign NaN if index is out of range

        # Get p-value from the normal distribution
        P_Value = 2 * (1 - stats.norm.cdf(np.abs(Z_Statistic)))
        
        # Append results to the list
        Results_List.append({'Variable': Variable, 'Z_Statistic': Z_Statistic, 'P_Value': P_Value})
    
    # Convert the results list to a DataFrame
    Results = pd.DataFrame(Results_List)
    
    return Results


In [407]:
# Ahora, usa el DataFrame normalizado para el test de Brant
Brant_Results = Brant_Test(Normalized_DataFrame, Variables_Dependientes[0], Variables_Independientes)

Optimization terminated successfully.
         Current function value: 1.147268
         Iterations 7


In [410]:
df.head(10)

,cercania_Massa,cercania_Milei,cercania_Bregman,cercania_Bullrich,cercania_Schiaretti,edad,autopercep_izqder,autopercep_conpro,autopercep_perantiper,indice_positividad,indice_progresismo,indice_conservadurismo,genero_femenino,genero_masculino,genero_Others,voto_2019_Alberto_Fernandez,voto_2019_Mauricio_Macri,voto_2019_Nicolás_Del_Caño,voto_2019_no_vote,voto_2019_Others
0,1,2,1,4,5,2.827345,0.886888,0.409636,1.862703,-0.098204,-0.259183,-0.304098,0.508579,0,0,-1.116761,2.288947,0,0,0
1,1,3,1,1,4,1.558093,1.859649,-2.223467,-0.594485,-0.098204,1.787203,-2.744637,-1.966265,1,0,-1.116761,-0.436882,0,0,1
2,1,1,1,5,1,2.924980,2.832410,1.287337,1.862703,-0.098204,1.056351,-2.917588,-1.966265,1,0,-1.116761,2.288947,0,0,0
3,1,1,1,3,1,1.167554,2.832410,-2.223467,1.862703,-0.715806,0.793244,-1.457108,0.508579,0,0,-1.116761,2.288947,0,0,0
4,1,3,1,5,3,-0.101697,0.886888,-0.906915,0.458596,-0.715806,0.267031,-0.496266,0.508579,0,0,-1.116761,2.288947,0,0,0
5,1,2,4,1,2,1.265189,-0.572254,1.287337,0.107569,-0.715806,-0.653843,0.272408,0.508579,0,0,-1.116761,-0.436882,1,0,0
6,2,1,4,1,1,-0.785140,-1.058634,0.409636,0.107569,-1.024608,0.033158,0.541444,0.508579,0,0,-1.116761,-0.436882,1,0,0
7,3,1,5,1,2,0.191207,-1.545015,1.287337,0.107569,-0.407005,-1.282375,1.060298,0.508579,0,0,0.895447,-0.436882,0,0,0
8,2,1,4,1,2,-0.687505,-1.545015,1.287337,-0.243458,1.445802,-0.697694,0.541444,-1.966265,1,0,-1.116761,-0.436882,0,0,1
9,1,1,1,1,1,-1.175679,0.400507,-0.029214,0.107569,0.210597,1.056351,-0.496266,-1.966265,1,0,-1.116761,-0.436882,0,1,0


In [408]:
# Mostrar los resultados del test de Brant.
print(Brant_Results)

                      Variable  Z_Statistic  P_Value
0                         edad   938.586211      0.0
1            autopercep_izqder  4152.838453      0.0
2            autopercep_conpro  3483.294676      0.0
3        autopercep_perantiper  2480.618494      0.0
4           indice_positividad  1778.520932      0.0
5           indice_progresismo          NaN      NaN
6       indice_conservadurismo          NaN      NaN
7              genero_femenino          NaN      NaN
8  voto_2019_Alberto_Fernandez          NaN      NaN
9     voto_2019_Mauricio_Macri          NaN      NaN
